# Vector Database 客户端库教程 (FAISS 重点)

欢迎来到向量数据库客户端库教程！随着嵌入（Embeddings）技术在表示文本、图像等非结构化数据方面的成功，**向量数据库**（或称向量搜索引擎）成为了现代 AI 应用的关键基础设施。它们专门用于存储高维向量，并能高效地执行**相似性搜索 (Similarity Search)**，即根据一个查询向量找出数据库中最相似的向量。

**为什么需要向量数据库/搜索库？**

1.  **高效检索**: 对于大规模向量数据集（百万甚至十亿级别），传统的线性扫描计算相似度非常慢。向量数据库使用近似最近邻 (Approximate Nearest Neighbor, ANN) 等算法来大幅加速搜索过程。
2.  **RAG 的核心**: 在检索增强生成 (RAG) 中，向量数据库用于存储文档块的嵌入向量，并根据用户问题的嵌入向量快速找到相关的文档块。
3.  **推荐系统**: 找到与用户或物品嵌入向量相似的其他用户或物品。
4.  **图像/音频搜索**: 基于内容的图像或音频检索。
5.  **重复数据删除**: 查找相似的文本或图像。

**本教程重点介绍 `faiss-cpu` (或 `faiss-gpu`)**: 
*   FAISS (Facebook AI Similarity Search) 是由 Facebook AI 开发的一个非常高效的向量相似性搜索库。
*   它提供了多种索引类型，可以在内存或磁盘上运行。
*   它是一个库，而不是一个数据库服务，通常嵌入在应用程序中或由其他框架（如 LangChain, LlamaIndex）调用。

**其他流行的向量数据库/库 (简介):**
*   **ChromaDB**: 开源，本地优先，易于使用，与 LangChain/LlamaIndex 集成良好。
*   **Pinecone**: 商业化的、完全托管的云原生向量数据库服务。
*   **Weaviate**: 开源的云原生向量数据库，支持 GraphQL。
*   **Milvus**: 开源的云原生向量数据库。

**本教程将涵盖 FAISS 的核心用法：**

1.  安装 FAISS
2.  准备示例向量数据 (使用 Sentence Transformers 获取文本嵌入)
3.  构建 FAISS 索引 (如 `IndexFlatL2`, `IndexIVFFlat`)
4.  向索引添加向量
5.  执行相似性搜索 (`index.search()`)
6.  (简介) 索引的保存与加载
7.  (简介) 与 LangChain/LlamaIndex 的集成

## 1. 安装 FAISS 和 Sentence Transformers

```bash
# 安装 FAISS CPU 版本
pip install faiss-cpu

# 或者，如果你有兼容的 GPU 和 CUDA 环境，可以安装 GPU 版本
# pip install faiss-gpu

# 安装 Sentence Transformers 用于生成文本嵌入
pip install sentence-transformers

# 其他依赖
pip install numpy pandas matplotlib
```

In [ ]:
import numpy as np
import time
import os

# 尝试导入 faiss
try:
    import faiss
    print(f"FAISS version: {faiss.__version__}")
    faiss_available = True
except ImportError:
    print("FAISS library not found. Please install faiss-cpu or faiss-gpu.")
    print("pip install faiss-cpu")
    faiss_available = False

# 尝试导入 sentence-transformers
try:
    from sentence_transformers import SentenceTransformer
    print("SentenceTransformer imported successfully.")
    st_available = True
    # 加载一个预训练的嵌入模型 (第一次运行时会自动下载)
    # 'all-MiniLM-L6-v2' 是一个常用且相对较小的模型
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    embedding_dim = embedding_model.get_sentence_embedding_dimension()
    print(f"Loaded Sentence Transformer model. Embedding dimension: {embedding_dim}")
except ImportError:
    print("sentence-transformers library not found. Please install it: pip install sentence-transformers")
    st_available = False
    embedding_model = None
    embedding_dim = None
except Exception as e:
    print(f"Error loading Sentence Transformer model: {e}. Check internet connection or model name.")
    st_available = False
    embedding_model = None
    embedding_dim = None

# 辅助函数
def time_it(func, *args, **kwargs):
    start = time.time()
    result = func(*args, **kwargs)
    end = time.time()
    print(f"Execution time: {end - start:.4f} seconds")
    return result

## 2. 准备示例向量数据

我们需要一组向量来构建索引。这里，我们使用 `sentence-transformers` 将一些示例文本转换为嵌入向量。

In [ ]:
print("--- Preparing Sample Data and Embeddings ---")

documents = [
    "The cat sat on the mat.",
    "The dog chased the ball.",
    "Apples and oranges are fruits.",
    "Paris is the capital of France.",
    "The weather is sunny today.",
    "Machine learning models require data.",
    "A feline rested upon the rug.", # Similar to sentence 1
    "Information retrieval is key for RAG."
]

doc_embeddings = None
if st_available and embedding_model:
    print(f"Generating embeddings for {len(documents)} documents...")
    # 使用 embedding_model.encode() 获取嵌入向量
    doc_embeddings = embedding_model.encode(documents)
    
    # FAISS 需要 float32 类型的 NumPy 数组
    doc_embeddings = np.array(doc_embeddings).astype('float32')
    
    print(f"Embeddings generated. Shape: {doc_embeddings.shape}") # (num_documents, embedding_dim)
    # print("Sample embedding (first 5 dims of first doc):")
    # print(doc_embeddings[0, :5])
else:
    print("Sentence Transformer model not available. Cannot generate embeddings.")

## 3. 构建 FAISS 索引

FAISS 提供了多种索引类型，适用于不同的数据规模、内存限制和搜索速度/精度权衡。

*   **`faiss.IndexFlatL2`**: 
    *   最简单的索引，进行精确的暴力 L2 (欧氏距离) 搜索。
    *   不需要训练。
    *   适用于小型数据集，精度最高，但速度最慢。
*   **`faiss.IndexFlatIP`**: 类似 `IndexFlatL2`，但使用内积 (Inner Product) 作为相似度度量（对于归一化向量，等价于余弦相似度）。
*   **`faiss.IndexIVFFlat`**: 
    *   基于倒排文件 (Inverted File) 的索引，是常用的近似最近邻 (ANN) 算法。
    *   需要一个**训练 (train)** 阶段，使用一部分数据（或全部数据）来学习数据空间中的聚类中心 (centroids)。
    *   搜索时，先找到查询向量最近的几个聚类中心，然后在这些中心对应的列表中进行搜索，从而减少搜索范围。
    *   参数：`quantizer` (通常是 `IndexFlatL2`)，`d` (向量维度)，`nlist` (聚类中心数量)。
    *   `nprobe` 参数控制搜索时要检查的聚类列表数量（影响速度和精度）。
*   **其他索引**: 如 `IndexPQ` (Product Quantization), `IndexHNSWFlat` (Hierarchical Navigable Small World graphs) 等，用于更大规模或需要更高压缩率/速度的场景。

In [ ]:
print("\n--- Building FAISS Index --- ")
index_flat_l2 = None
index_ivf_flat = None

if faiss_available and doc_embeddings is not None:
    d = embedding_dim # 向量维度
    n_docs = doc_embeddings.shape[0]
    
    # --- 1. IndexFlatL2 (精确，暴力搜索) --- 
    print("\nBuilding IndexFlatL2...")
    index_flat_l2 = faiss.IndexFlatL2(d)
    print(f"IndexFlatL2 created. Is trained: {index_flat_l2.is_trained}")
    print(f"Initial vector count: {index_flat_l2.ntotal}")
    
    # --- 2. IndexIVFFlat (近似，需要训练) --- 
    print("\nBuilding IndexIVFFlat...")
    nlist = 4 # 聚类中心数量 (通常选择 sqrt(N) 到 N/100 之间，这里 N 很小)
    quantizer = faiss.IndexFlatL2(d) # 底层使用 L2 距离计算中心
    index_ivf_flat = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
    # faiss.METRIC_L2: 使用 L2 距离
    # faiss.METRIC_INNER_PRODUCT: 使用内积
    
    print(f"IndexIVFFlat created. Is trained: {index_ivf_flat.is_trained}")
    
    # 训练索引 (学习聚类中心)
    if n_docs >= nlist: # Need enough data to train
      print(f"Training IndexIVFFlat with {n_docs} vectors...")
      time_it(index_ivf_flat.train, doc_embeddings)
      print(f"IndexIVFFlat trained: {index_ivf_flat.is_trained}")
    else:
      print(f"Skipping IndexIVFFlat training (need at least {nlist} vectors, got {n_docs}).")
      index_ivf_flat = None # Cannot use untrained IVF index
      
else:
    print("FAISS or embeddings not available, cannot build index.")

## 4. 向索引添加向量

使用索引对象的 `.add(vectors)` 方法将向量（必须是 float32 NumPy 数组）添加到索引中。
对于某些索引（如 IVF），向量会被分配到最近的聚类中心对应的列表中。

In [ ]:
print("\n--- Adding Vectors to Index --- ")

if index_flat_l2 and doc_embeddings is not None:
    print("Adding vectors to IndexFlatL2...")
    index_flat_l2.add(doc_embeddings)
    print(f"IndexFlatL2 vector count: {index_flat_l2.ntotal}")

if index_ivf_flat and index_ivf_flat.is_trained and doc_embeddings is not None:
    print("\nAdding vectors to IndexIVFFlat...")
    index_ivf_flat.add(doc_embeddings)
    print(f"IndexIVFFlat vector count: {index_ivf_flat.ntotal}")

## 5. 执行相似性搜索 (`index.search()`)

使用 `.search(query_vectors, k)` 方法来查找与查询向量最相似的 `k` 个向量。

*   `query_vectors`: 一个包含一个或多个查询向量的 float32 NumPy 数组 (形状 `[num_queries, dimension]`)。
*   `k`: 要查找的最近邻的数量。
*   返回两个数组：
    *   `D`: 距离数组 (形状 `[num_queries, k]`)，包含查询向量到每个最近邻的距离（L2 距离或负内积）。
    *   `I`: 索引数组 (形状 `[num_queries, k]`)，包含每个最近邻在原始添加数据中的索引。

In [ ]:
print("\n--- Performing Similarity Search --- ")

if (index_flat_l2 or index_ivf_flat) and embedding_model:
    query_text = " feline animal " # Query related to 'cat sat on the mat'
    k = 3 # Find top 3 similar documents
    
    print(f"Query text: '{query_text}'")
    query_embedding = embedding_model.encode([query_text]).astype('float32')
    print(f"Query embedding shape: {query_embedding.shape}")
    
    # --- Search using IndexFlatL2 --- 
    if index_flat_l2 and index_flat_l2.ntotal > 0:
        print("\nSearching using IndexFlatL2...")
        distances_flat, indices_flat = time_it(index_flat_l2.search, query_embedding, k)
        print(f"  Distances (L2): {distances_flat[0]}")
        print(f"  Indices: {indices_flat[0]}")
        print("  Retrieved Documents (FlatL2):")
        for i, idx in enumerate(indices_flat[0]):
            if 0 <= idx < len(documents):
                 print(f"    {i+1}. Index={idx}, Dist={distances_flat[0][i]:.4f} - '{documents[idx]}'")
            else:
                 print(f"    {i+1}. Invalid index {idx} found.")
                 
    # --- Search using IndexIVFFlat --- 
    if index_ivf_flat and index_ivf_flat.is_trained and index_ivf_flat.ntotal > 0:
        index_ivf_flat.nprobe = 2 # Search in the 2 nearest clusters (adjust for speed/accuracy)
        print(f"\nSearching using IndexIVFFlat (nprobe={index_ivf_flat.nprobe})...")
        distances_ivf, indices_ivf = time_it(index_ivf_flat.search, query_embedding, k)
        print(f"  Distances (L2): {distances_ivf[0]}")
        print(f"  Indices: {indices_ivf[0]}")
        print("  Retrieved Documents (IVFFlat):")
        # Note: Indices might be -1 if fewer than k results are found in probed clusters
        for i, idx in enumerate(indices_ivf[0]):
            if idx != -1 and 0 <= idx < len(documents):
                 print(f"    {i+1}. Index={idx}, Dist={distances_ivf[0][i]:.4f} - '{documents[idx]}'")
            elif idx == -1:
                 print(f"    {i+1}. No result found in probed clusters for this rank.")
            else:
                 print(f"    {i+1}. Invalid index {idx} found.")
else:
    print("Index or embedding model not available, skipping search example.")

## 6. (简介) 索引的保存与加载

FAISS 索引可以保存到磁盘，以便后续重用，避免重新构建。

```python
# import faiss

# index_to_save = index_flat_l2 # Or index_ivf_flat
# index_filename = "my_faiss_index.index"

# # --- 保存 --- 
# if index_to_save:
#     print(f"Saving index to {index_filename}...")
#     faiss.write_index(index_to_save, index_filename)
#     print("Index saved.")

# # --- 加载 --- 
# if os.path.exists(index_filename):
#     print(f"\nLoading index from {index_filename}...")
#     loaded_index = faiss.read_index(index_filename)
#     print(f"Index loaded. Vector count: {loaded_index.ntotal}")
      # Ready to use loaded_index.search(...)
#     os.remove(index_filename) # Cleanup
# else:
#     print("Index file not found for loading.")
```

## 7. (简介) 与 LangChain/LlamaIndex 的集成

FAISS 可以作为 LangChain 和 LlamaIndex 的向量存储后端。

**LangChain 示例:**
```python
# from langchain.vectorstores import FAISS
# from langchain.embeddings import OpenAIEmbeddings # Or other embeddings
# from langchain.docstore.document import Document

# # Assuming 'split_docs' is a list of LangChain Document objects
# embeddings = OpenAIEmbeddings()
# vectorstore_lc = FAISS.from_documents(split_docs, embeddings)
# retriever_lc = vectorstore_lc.as_retriever()
# results = retriever_lc.get_relevant_documents("your query")
# vectorstore_lc.save_local("faiss_langchain_index")
# loaded_vectorstore_lc = FAISS.load_local("faiss_langchain_index", embeddings)
```

**LlamaIndex 示例:**
```python
# from llama_index.vector_stores import FaissVectorStore
# from llama_index import VectorStoreIndex, StorageContext
# import faiss # Need faiss installed

# # Assuming 'nodes' is a list of LlamaIndex Node objects
# # Assuming Settings.embed_model is configured

# # 1. Create FAISS index directly
# d = Settings.embed_model.embed_dim 
# faiss_index = faiss.IndexFlatL2(d)

# # 2. Create FaissVectorStore wrapper
# vector_store_li = FaissVectorStore(faiss_index=faiss_index)

# # 3. Create StorageContext and build index
# storage_context = StorageContext.from_defaults(vector_store=vector_store_li)
# index_li = VectorStoreIndex(nodes, storage_context=storage_context)

# # Or, let LlamaIndex handle FAISS creation internally during VectorStoreIndex build
# # (This might happen if you don't explicitly provide a vector_store)

# # Persisting is usually done via the StorageContext
# # index_li.storage_context.persist(persist_dir="./faiss_llamaindex_index")

# # Loading
# # from llama_index import load_index_from_storage, StorageContext
# # storage_context_load = StorageContext.from_defaults(persist_dir="./faiss_llamaindex_index")
# # loaded_index_li = load_index_from_storage(storage_context_load)
```
通常，使用 LangChain 或 LlamaIndex 提供的封装会更方便，它们会处理好索引构建、添加和搜索的细节。

## 总结

FAISS 是一个用于高效向量相似性搜索的强大库，是构建现代 AI 应用（尤其是 RAG 系统）的关键组件。

**关键要点：**
*   向量数据库/索引库用于存储和快速检索高维嵌入向量。
*   FAISS 提供了多种索引类型，需要在速度、内存和精度之间进行权衡 (`IndexFlatL2` 精确但慢, `IndexIVFFlat` 等 ANN 索引更快但近似)。
*   核心操作包括构建索引 (`faiss.Index...`)、训练索引 (如果需要)、添加向量 (`.add()`) 和搜索 (`.search()`)。
*   FAISS 需要 NumPy float32 数组作为输入。
*   通常与文本嵌入模型 (如 Sentence Transformers) 结合使用。
*   可以作为 LangChain 和 LlamaIndex 的向量存储后端，通常由这些框架封装其使用细节。

理解向量搜索的基本原理以及 FAISS 等库的核心用法，对于构建和优化基于嵌入的 AI 应用非常有帮助。